In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy import stats
import krippendorff
import matplotlib.pyplot as plt

In [4]:
path = "experiments/done/new_promptvariation_bias"
results_df = pd.read_parquet(path+"/results.parquet")

print(len(results_df))
results_df["llm_decision"].value_counts()
print(len(results_df["combination_id"].unique()))
# results_df.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)
results_df

12960
432


,experiment_id,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,reasoning_length,processing_time,total_tokens,tokens_per_second,final_prompt_type_category,final_prompt_type_value,think_mode_category,think_mode_value,bias_category,bias_value
0,new_promptvariation_bias,0,0,8723,Check-Out,Check-Out,Check-Out,0,0.713890,1668,2336.495731,ai_neutral_emotional,Feel the electric tension crackling in the air...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,avoid_fn,A false negative may result in lost opportunit...
1,new_promptvariation_bias,0,1,10233,Check-Out,Check-Out,Check-Out,0,0.255827,1646,6434.036756,ai_neutral_emotional,Feel the electric tension crackling in the air...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,avoid_fn,A false negative may result in lost opportunit...
2,new_promptvariation_bias,0,2,52028,Cancellation,Cancellation,Cancellation,0,0.250824,1625,6478.653196,ai_neutral_emotional,Feel the electric tension crackling in the air...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,avoid_fn,A false negative may result in lost opportunit...
3,new_promptvariation_bias,0,3,16948,Check-Out,Cancellation,Cancellation,0,0.257941,1610,6241.748806,ai_neutral_emotional,Feel the electric tension crackling in the air...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,avoid_fn,A false negative may result in lost opportunit...
4,new_promptvariation_bias,0,4,16376,Check-Out,Check-Out,Check-Out,0,0.252794,1608,6360.921775,ai_neutral_emotional,Feel the electric tension crackling in the air...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,avoid_fn,A false negative may result in lost opportunit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,new_promptvariation_bias,431,25,84583,Check-Out,Check-Out,Check-Out,0,0.239837,1605,6692.033628,ai_neutral_rational,Given the hotel booking data and AI prediction...,cot,Approach this methodically: assess the AI’s ou...,None,None
12956,new_promptvariation_bias,431,26,96373,Check-Out,Cancellation,Cancellation,1773,2.720095,1982,728.650979,ai_neutral_rational,Given the hotel booking data and AI prediction...,cot,Approach this methodically: assess the AI’s ou...,None,None
12957,new_promptvariation_bias,431,27,99102,Check-Out,Check-Out,Check-Out,0,0.221644,1616,7290.957898,ai_neutral_rational,Given the hotel booking data and AI prediction...,cot,Approach this methodically: assess the AI’s ou...,None,None
12958,new_promptvariation_bias,431,28,106685,Check-Out,Check-Out,Check-Out,0,0.264714,1627,6146.261434,ai_neutral_rational,Given the hotel booking data and AI prediction...,cot,Approach this methodically: assess the AI’s ou...,None,None


In [50]:
# path = "experiments/prompt_variations_exclusion_shap-lime"
# results_df2 = pd.read_parquet(path+"/results.parquet")
# results_df2.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)
#
# print(results_df2["final_prompt_type_category"].unique())
# results_df2["combination_id"] = results_df2["combination_id"].apply(lambda x: x+100)
# results_df2["xai_exclusion_pdp"] = False
# results_df2["xai_exclusion_histogram"] = False
#
#
# results_df["xai_exclusion_shap"] =False
# results_df["xai_exclusion_lime_explanation"] = False
# results_df = pd.concat([results_df, results_df2], ignore_index=True)
# results_df

In [16]:
# results_df["method_order"] = results_df["method_order"].apply(lambda x: str(x)[1:-1])
# results_df

In [4]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary


columns = summary.columns
for column in columns:
    if column in ['n_datapoints', 'percent_correct', 'percent_agree', 'corr_correct',
       'corr_agree', 'false_positive_rate', 'false_negative_rate',"think_mode_value"]:
        continue

    marginal = column#"final_prompt_type_category"
    comparison = analyzer.compare_parameters(marginal,True)
    comparison = comparison.round(4).drop(["n_combinations"], axis=1)
    comparison


    df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
    df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                             "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                             "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                             "avg_false_positive_rate": "$\\overline{FP}$",
                             "final_prompt_type_category":"Prompt category"}, inplace=True)
    table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                            column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
    table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
    print(table_str)
    print()

#\hspace*{-1.5cm}


# df_t = df_clean.T
# df_t = df_t.set_axis(df_t.iloc[0], axis=1).iloc[1:]
# df_t.columns = df_t.columns.str.replace('_', ' ')
#
# print(df_t.to_latex(escape=True, float_format="%.2f", caption=marginal.replace("_", " "),
#                                                              column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c"))



\begin{table}
\caption{final prompt type category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
Prompt category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
ai neutral emotional & 0.47 & 68.66+-4.47 & 79.27+-10.58 & 0.30+-0.12 & 0.58+-0.22 & 0.22+-0.07 & 0.09+-0.07 \\
ai neutral rational & 0.62 & 69.43+-4.21 & 84.71+-8.08 & 0.33+-0.11 & 0.70+-0.16 & 0.18+-0.06 & 0.12+-0.05 \\
\bottomrule
\end{tabular}
\end{table}


\begin{table}
\caption{think mode category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
think mode category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{

In [53]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
comparison = analyzer.compare_parameters(None,True)
comparison = comparison.round(4).drop(["n_combinations"], axis=1)
comparison


df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                         "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                         "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                         "avg_false_positive_rate": "$\\overline{FP}$",
                         "final_prompt_type_category":"Prompt category"}, inplace=True)
table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                        column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
print(table_str)


\begin{table}
\caption{xai exclusion histograms}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
$\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
0.51 & 67.67+-7.17 & 84.50+-14.03 & 0.28+-0.15 & 0.68+-0.30 & 0.20+-0.07 & 0.12+-0.09 \\
\bottomrule
\end{tabular}
\end{table}



In [3]:
import pandas as pd

data = {
    'XAI Method': ['SHAP', 'SHAP', 'PDP', 'PDP', 'LIME', 'LIME', 'Counterfactual', 'Counterfactual', 'Histograms', 'Histograms', 'All'],
    'Exclusion': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes'],
    'α': [0.56, 0.68, 0.65, 0.59, 0.66, 0.59, 0.63, 0.62, 0.61, 0.62, 0.51],
    'Avg. % correct': [67.05, 67.64, 67.66, 67.04, 67.81, 66.88, 66.29, 68.40, 67.35, 67.34, 67.67],
    'correct_std': [5.79, 4.98, 4.65, 6.06, 5.00, 5.75, 5.64, 4.95, 5.11, 5.70, 7.17],
    'Avg. % agree': [85.51, 90.48, 88.85, 87.14, 89.44, 86.56, 87.46, 88.54, 87.95, 88.05, 84.50],
    'agree_std': [12.48, 10.28, 10.35, 12.86, 10.75, 12.41, 11.38, 12.00, 11.57, 11.84, 14.03],
    'r_correct': [0.27, 0.30, 0.28, 0.28, 0.30, 0.27, 0.25, 0.31, 0.28, 0.28, 0.28],
    'r_correct_std': [0.15, 0.11, 0.13, 0.13, 0.12, 0.14, 0.14, 0.11, 0.12, 0.14, 0.15],
    'r_agree': [0.70, 0.81, 0.77, 0.74, 0.79, 0.73, 0.74, 0.77, 0.76, 0.76, 0.68],
    'r_agree_std': [0.26, 0.20, 0.22, 0.26, 0.21, 0.26, 0.24, 0.24, 0.24, 0.24, 0.30],
    'FP_mean': [0.20, 0.18, 0.19, 0.19, 0.18, 0.19, 0.20, 0.18, 0.19, 0.19, 0.20],
    'FP_std': [0.06, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07],
    'FN_mean': [0.13, 0.15, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.12],
    'FN_std': [0.06, 0.06, 0.04, 0.07, 0.06, 0.06, 0.05, 0.07, 0.06, 0.06, 0.09]
}

df = pd.DataFrame(data)
df

,XAI Method,Exclusion,α,Avg. % correct,correct_std,Avg. % agree,agree_std,r_correct,r_correct_std,r_agree,r_agree_std,FP_mean,FP_std,FN_mean,FN_std
0,SHAP,No,0.56,67.05,5.79,85.51,12.48,0.27,0.15,0.70,0.26,0.20,0.06,0.13,0.06
1,SHAP,Yes,0.68,67.64,4.98,90.48,10.28,0.30,0.11,0.81,0.20,0.18,0.05,0.15,0.06
2,PDP,No,0.65,67.66,4.65,88.85,10.35,0.28,0.13,0.77,0.22,0.19,0.06,0.14,0.04
3,PDP,Yes,0.59,67.04,6.06,87.14,12.86,0.28,0.13,0.74,0.26,0.19,0.06,0.14,0.07
4,LIME,No,0.66,67.81,5.00,89.44,10.75,0.30,0.12,0.79,0.21,0.18,0.06,0.14,0.06
5,LIME,Yes,0.59,66.88,5.75,86.56,12.41,0.27,0.14,0.73,0.26,0.19,0.06,0.14,0.06
6,Counterfactual,No,0.63,66.29,5.64,87.46,11.38,0.25,0.14,0.74,0.24,0.20,0.06,0.14,0.05
7,Counterfactual,Yes,0.62,68.40,4.95,88.54,12.00,0.31,0.11,0.77,0.24,0.18,0.06,0.14,0.07
8,Histograms,No,0.61,67.35,5.11,87.95,11.57,0.28,0.12,0.76,0.24,0.19,0.06,0.14,0.06
9,Histograms,Yes,0.62,67.34,5.70,88.05,11.84,0.28,0.14,0.76,0.24,0.19,0.06,0.14,0.06


In [5]:
df_no = df[df['Exclusion'] == 'No']
df_no["Krippendorffs Alpha"].mean()

np.float64(0.6224000000000001)